In [1]:
import numpy as np
import pandas as pd
# data=np.load("first_batch_with_labels_likes.npz")
# data2=np.load("second_batch_with_labels_likes.npz")
data = np.load("fifth_batch_with_labels_likes.npz")

In [2]:
X=data["X"]
y=data["y"]

XX=pd.DataFrame(X)
yy=pd.DataFrame(y)

# X2=data2["X"]
# y2=data2["y"]

# XX2=pd.DataFrame(X2)
# yy2=pd.DataFrame(y2)

XX.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)
# XX2.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)

# XX = pd.concat([XX, XX2])
# XX = XX.sort_values(by=["user"], ascending=True)
# XX = XX.reset_index(drop=True)

In [9]:
XX

,user,item,rating
134809,5100,211,3
134811,5100,219,3
134801,5100,141,2
134789,5100,45,2
134815,5100,243,1
...,...,...,...
72480,7099,862,3
72483,7099,928,2
72485,7099,934,3
72472,7099,388,3


In [17]:
XX.groupby('user')['item'].agg('mean')

user
5100    125.277778
5101    514.985915
5102    472.320442
5103    379.952381
5104    194.918033
           ...    
7095    442.095238
7096    383.105263
7097    479.477612
7098    428.131579
7099    593.142857
Name: item, Length: 2000, dtype: float64

In [4]:
yy.rename(columns={0:"user",1:"label"},inplace=True)
# yy2.rename(columns={0:"user",1:"label"},inplace=True)

# yy = pd.concat([yy,yy2], ignore_index=True)
yy

,user,label
0,5100,0
1,5101,1
2,5102,1
3,5103,0
4,5104,0
...,...,...
1995,7095,0
1996,7096,0
1997,7097,1
1998,7098,1


In [5]:
sm = pd.DataFrame(0, index=range(len(set(XX['user']))), columns=range(XX['item'].max()+1))
sm.index = list(set(XX.user))
sm.sort_index()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
5100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
XX = XX.sort_values(by=["item"], ascending=True)
dictItem = {}
for i, row in XX.iterrows():
    rating = row["rating"]
    item = row["item"]

    try:
        keyValue = dictItem[item]
    except KeyError:
        keyValue = [0,0,0]
    if(rating == 1):
        keyValue[0] = keyValue[0] + 1
        dictItem[item] = keyValue

    elif(rating == 0):
        keyValue[1] = keyValue[1] + 1
        dictItem[item] = keyValue

    elif(rating == -1):
        keyValue[2] = keyValue[2] + 1
        dictItem[item] = keyValue
        
for i in dictItem:
    itemValue = dictItem[i]
    largestIdx = itemValue.index(max(itemValue))
    if(largestIdx == 0):
        dictItem[i] = 1
    if(largestIdx == 1):
        dictItem[i] = 0
    if(largestIdx == 2):
        dictItem[i] = -1

XX = XX.sort_values(by=["user"], ascending=True)
curr_user = -1
userItem = {}
for index, row in XX.iterrows():
    if (row["user"] != curr_user):
        curr_user = row["user"]
        userItem[curr_user] = 0

    mostPopRating = dictItem[row["item"]]

    if(mostPopRating == row["rating"]):
        userItem[curr_user] = userItem[curr_user] + 1

firstKey = list(userItem.keys())[0]

userItemCol = pd.DataFrame(userItem.values())
userItemCol.index = pd.RangeIndex(firstKey,firstKey + len(userItemCol))

In [7]:
# Ratings will now be, 0 -> didn't rate, 1 -> dislike, 2 -> meh, 3 -> like
XX["rating"] = XX["rating"].replace(1, 3)
XX["rating"] = XX["rating"].replace(0, 2)
XX["rating"] = XX["rating"].replace(-1, 1)
curr_user = -1

for index, row in XX.iterrows():
    if (row["user"] != curr_user):
        curr_user = row["user"]
    sm.loc[int(curr_user), int(row["item"])] = row["rating"]

In [8]:
sm

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
5100,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5101,0,0,0,0,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
5102,3,0,0,0,0,0,3,2,0,0,...,0,0,0,0,0,0,0,0,1,0
5103,3,2,2,0,0,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
5104,3,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,2,1,0,1,3,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
7096,0,1,0,0,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
7097,3,0,0,0,3,2,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
7098,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
sm['user'] = sm.index
sm = sm.sort_index()
sm

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,user
5100,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5100
5101,0,0,0,0,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,5101
5102,3,0,0,0,0,0,3,2,0,0,...,0,0,0,0,0,0,0,1,0,5102
5103,3,2,2,0,0,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,5103
5104,3,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,2,1,0,1,3,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,7095
7096,0,1,0,0,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,7096
7097,3,0,0,0,3,2,3,3,0,0,...,0,0,0,0,0,0,0,0,0,7097
7098,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7098


In [27]:
df_grouped = XX.groupby('user').agg(
    total_interactions=('rating', 'size'),
    likes=('rating', lambda x: (x == 3).sum()),
    dislikes=('rating', lambda x: (x == 1).sum()),
    meh=('rating', lambda x: (x == 2).sum())
)

df_grouped['mean'] = df_grouped['likes'] + df_grouped['dislikes'] + df_grouped['meh'] / 3
df_grouped['std'] = df_grouped[['likes', 'dislikes', 'meh']].std(axis=1)
df_grouped['cv'] = df_grouped['std']/df_grouped['mean'] * 100

df_grouped['followed majority'] = userItemCol
df_grouped['followed majority %'] = df_grouped['followed majority'] / df_grouped['total_interactions']
# df_grouped.drop(['user'], axis=1, inplace=True)

# Uncomment this if you would like to drop some features r
# df_grouped = df_grouped.drop(columns=['total_interactions', 'likes', 'dislikes', 'meh', 'mean', 'std'])
df_grouped

# Merging with labels to create a single DataFrame
df_final = pd.concat([sm, df_grouped], axis=1)
# df_final = df_final.merge(yy)

df_final

,0,1,2,3,4,5,6,7,8,9,...,user,total_interactions,likes,dislikes,meh,mean,std,cv,followed majority,followed majority %
5100,2,0,0,1,0,0,0,0,1,0,...,5100,36,16,6,14,26.666667,5.291503,19.843135,20,0.555556
5101,0,0,0,0,1,0,0,3,0,0,...,5101,71,21,28,22,56.333333,3.785939,6.720602,24,0.338028
5102,3,0,0,0,0,0,3,2,0,0,...,5102,181,123,5,53,145.666667,59.340823,40.737407,118,0.651934
5103,3,2,2,0,0,2,0,0,3,0,...,5103,294,184,28,82,239.333333,79.221209,33.100784,178,0.605442
5104,3,0,0,0,3,0,0,0,0,0,...,5104,61,53,2,6,57.000000,28.360771,49.755739,48,0.786885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,2,1,0,1,3,0,2,3,0,1,...,7095,504,243,58,203,368.666667,97.339612,26.403150,319,0.632937
7096,0,1,0,0,2,0,0,0,0,2,...,7096,133,29,35,69,87.000000,21.571586,24.794927,54,0.406015
7097,3,0,0,0,3,2,3,3,0,0,...,7097,201,111,23,67,156.333333,44.000000,28.144989,125,0.621891
7098,3,0,0,0,0,0,0,0,0,0,...,7098,76,25,30,21,62.000000,4.509250,7.272983,26,0.342105


In [34]:
from datasetStuff import expandDataset

data=np.load("fifth_batch_with_labels_likes.npz")
# data2=np.load("second_batch_with_labels_likes.npz")
df = expandDataset(data, True).getDataset()
df.index = df['user']
# df2 = expandDataset(data2, True).getDataset()

In [13]:
data=np.load("fifth_batch_with_labels_likes.npz")
df_more, labels_more = expandDataset(data).simpleDataRetrieval(data)

In [14]:
from stuff import stuff

df_more = stuff().doStuff(df_more)

In [15]:
df_more.index = df_more['user']

In [35]:
df = df.drop(list(set(df_final.columns)&set(df.columns)),axis=1)

In [36]:
df_more = df_more.drop(list(set(df_more.columns)&set(df_final.columns)),axis=1)

In [40]:
df_final = df_final.merge(df_more, left_index=True, right_index=True)

In [41]:
df_final = df_final.merge(df, left_index=True, right_index=True)

In [42]:
df_final = df_final.drop(['rating_diff'],axis=1)

In [43]:
df_final

,0,1,2,3,4,5,6,7,8,9,...,pythagoras_ldm,pythagoras_ratios,log_ldm,pyth_log_ratios,follow_rat_val,plr_frv,cv_foll,log_ratios,rating_ratio,pyth_log_ldm_ratios
5100,2,0,0,1,0,0,0,0,1,0,...,22.090722,3.723127,7.203406,26.819197,0.000000,0.000000,0.000000,1.961659,0.036108,0.000000
5101,0,0,0,0,1,0,0,3,0,0,...,41.340053,2.120169,9.467769,20.073275,0.000377,0.007571,0.094656,0.575364,0.071214,0.002327
5102,3,0,0,0,0,0,3,2,0,0,...,134.026117,26.886909,10.391914,279.406456,0.000081,0.022617,0.225069,6.405493,0.181545,0.001003
5103,3,2,2,0,0,2,0,0,3,0,...,203.381415,7.536266,12.953860,97.623728,0.000000,0.000000,0.000000,3.765462,0.294885,0.000000
5104,3,0,0,0,3,0,0,0,0,0,...,53.376025,28.094088,6.455199,181.352916,0.000762,0.138223,0.815668,6.554289,0.061184,0.008428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,2,1,0,1,3,0,2,3,0,1,...,321.903712,5.588929,14.866710,83.088987,0.002973,0.246992,16.711517,2.865237,0.505517,0.033433
7096,0,1,0,0,2,0,0,0,0,2,...,82.625662,3.317265,11.156750,37.009899,0.005968,0.220867,10.067113,1.733621,0.133400,0.051178
7097,3,0,0,0,3,2,3,3,0,0,...,131.677637,5.875512,12.049717,70.798263,0.007543,0.533999,17.503103,3.148072,0.201605,0.077590
7098,3,0,0,0,0,0,0,0,0,0,...,44.339599,2.213154,9.664596,21.389237,0.008707,0.186228,2.488126,0.713350,0.076229,0.056741


In [44]:
df_final.to_csv("./csv_files/fifth_expanded_sparse.csv")

In [24]:
data = np.load("fourth_batch_with_labels_likes.npz")
y=data["y"]
yy=pd.DataFrame(y)
yy.rename(columns={0:"user",1:"label"},inplace=True)

,user,label
0,3900,0
1,3901,1
2,3902,0
3,3903,0
4,3904,0
...,...,...
1195,5095,0
1196,5096,1
1197,5097,0
1198,5098,0


In [36]:
fourth = pd.read_csv("./csv_files/fourth_batch_to_predict.csv")

In [37]:
fourth = fourth.drop(['Unnamed: 0'],axis=1)
yy.index = yy['user']
fourth.index = fourth['user']
yy = yy.drop(['user'],axis=1)
fourht = fourth.drop(['user'],axis=1)
merged = fourth.merge(yy, left_index=True, right_index=True)
merged = merged.reset_index(drop=True)

In [55]:
merged.to_csv("./csv_files/fourth_expanded_sparse.csv")

In [58]:
f_s = pd.read_csv("./csv_files/f+s_expanded_sparse.csv")
third = pd.read_csv("./csv_files/third_expanded_sparse.csv")
fourth = pd.read_csv("./csv_files/fourth_expanded_sparse.csv")

In [59]:
f_s = f_s.drop(["Unnamed: 0"],axis=1)
third = third.drop(["Unnamed: 0"],axis=1)
fourth = fourth.drop(["Unnamed: 0"],axis=1)

In [63]:
third.index = third['user']
fourth.index = fourth['user']

In [68]:
all_merged = pd.concat([f_s,third,fourth])

In [69]:
all_merged.to_csv("./csv_files/f_s_t_f_expanded_sparse.csv")